# 1. Setup

In [1]:
%load_ext watermark

import glob
import datetime
import numpy as np
import pandas as pd
import xarray as xr

%watermark -iv -co -v

Python implementation: CPython
Python version       : 3.12.10
IPython version      : 9.2.0

conda environment: cpl_ppe_co2

numpy : 2.0.1
xarray: 2025.4.0
pandas: 2.2.2
sys   : 3.12.10 | packaged by conda-forge | (main, Apr 10 2025, 22:21:13) [GCC 13.3.0]



# 2. Define configuration and path management

In [2]:
# Claire's source code to check versions against
claire_srcdir = '/glade/u/home/czarakas/cesm_source/cesm_coupled_PPEn11'
print(f"Claire's CESM source code: {claire_srcdir}")
print('CESM tag:', end=' ')
!cd $claire_srcdir && git describe --tags
print('CLM  tag:', end=' ')
!cd $claire_srcdir/components/clm && git describe --tags
print('CAM  tag:', end=' ')
!cd $claire_srcdir/components/cam && git describe --tags

# Source code to use to run model
my_srcdir = '/glade/u/home/bbuchovecky/cesm_source/cesm_coupled_PPEn11'
print(f"\nmy CESM source code: {my_srcdir}")
print('CESM tag:', end=' ')
!cd $my_srcdir && git describe --tags
print('CLM  tag:', end=' ')
!cd $my_srcdir/components/clm && git describe --tags
print('CAM  tag:', end=' ')
!cd $my_srcdir/components/cam && git describe --tags

# Where you want to put the new folder for the case (where you change things like the 
# namelist, how long to run it, then do case.build, case.submit etc.)
# my_casedir = '/glade/u/home/bbuchovecky/cesm_runs/cases'
my_casedir = '/glade/u/home/bbuchovecky/cesm_runs/cases/cpl_ppe_co2/test_simulations'

# Where the output of the runs should be archived
# my_archivedir = '/glade/derecho/scratch/bbuchovecky/archive'
my_archivedir = '/glade/derecho/scratch/bbuchovecky/archive/cpl_ppe_co2/test_simulations'

# Where to put the run folder (where log files will go, etc.)
my_rundir = '/glade/derecho/scratch/bbuchovecky'

# Where you want to save the runscripts that this script will generate
# this can be anything, just make sure the folder exists!
runscript_path = '/glade/u/home/bbuchovecky/projects/cpl_ppe_co2/hist/code/03_run-scripts'

# Where to find ensemble member parameter files
paramfile_path = '/glade/u/home/bbuchovecky/projects/cpl_ppe_co2/hist/data/paramfiles'

# Where to find ensemble member namelist files
namelistmod_path = '/glade/u/home/bbuchovecky/projects/cpl_ppe_co2/hist/data/namelist_mods'

# The template to use to make the runscripts
template_file = './runscript.TEMPLATE'

print(f'\nCASE_DIR:    {my_casedir}')
print(f'ARCHIVE_DIR: {my_archivedir}')
print(f'RUN_DIR:     {my_rundir}')

Claire's CESM source code: /glade/u/home/czarakas/cesm_source/cesm_coupled_PPEn11
CESM tag: cesm2.2.0
CLM  tag: branch_tags/PPE.n11_ctsm5.1.dev030
CAM  tag: cam_cesm2_2_rel_02

my CESM source code: /glade/u/home/bbuchovecky/cesm_source/cesm_coupled_PPEn11
CESM tag: cesm2.2.0
CLM  tag: branch_tags/PPE.n11_ctsm5.1.dev030
CAM  tag: cam_cesm2_2_rel_02

CASE_DIR:    /glade/u/home/bbuchovecky/cesm_runs/cases/cpl_ppe_co2/test_simulations
ARCHIVE_DIR: /glade/derecho/scratch/bbuchovecky/archive/cpl_ppe_co2/test_simulations
RUN_DIR:     /glade/derecho/scratch/bbuchovecky


# 3. Brief check of parameter files

In [5]:
p0 = xr.open_dataset("/glade/u/home/czarakas/coupled_PPE/data/paramfiles/OAAT0000.nc", decode_timedelta=False)
p1 = xr.open_dataset("/glade/u/home/bbuchovecky/projects/cpl_ppe_co2/hist/data/paramfiles/COUP0000.nc", decode_timedelta=False)
p2 = xr.open_dataset("/glade/u/home/bbuchovecky/projects/cpl_ppe_co2/hist/data/paramfiles/coupPPE-hist.000.nc", decode_timedelta=False)

In [15]:
for var in p1.variables:
    if var in p0.variables:
        # Check if variable is string type
        if p1[var].dtype.kind in ['U', 'S', 'O']:  # Unicode, byte string, or object arrays
            are_equal = np.array_equal(p1[var].values, p0[var].values)
        else:
            are_equal = np.array_equal(p1[var].values, p0[var].values, equal_nan=True)
        
        if not are_equal:
            print(f"{var}: {are_equal}")
    else:
        print(f"{var}: not found in p0")

In [19]:
for var in p1.variables:
    if var in p2.variables:
        # Check if variable is string type
        if p1[var].dtype.kind in ['U', 'S', 'O']:  # Unicode, byte string, or object arrays
            are_equal = np.array_equal(p1[var].values, p2[var].values)
        else:
            are_equal = np.array_equal(p1[var].values, p2[var].values, equal_nan=True)
        
        if not are_equal:
            print(f"{var}: {are_equal}")
    else:
        print(f"{var}: not found in p2")

In [17]:
var = "pftname"
if p1[var].dtype.kind in ['U', 'S', 'O']:  # Unicode, byte string, or object arrays
    are_equal = np.array_equal(p1[var].values, p0[var].values)
else:
    are_equal = np.array_equal(p1[var].values, p0[var].values, equal_nan=True)
print(f"{var} arrays are equal: {are_equal}")

pftname arrays are equal: True


# 4. Choose naming convention

In [2]:
n = 0

nstring = str(n).zfill(3)
cmps = "FHIST_BGC"
lcmps = cmps[0].lower()
vcesm = "e21"
res = "f19_f19_mg17"
prefix = f"{lcmps}.{vcesm}.{cmps}.{res}"

datetag = datetime.datetime.now().strftime('%Y-%m-%d')

print("MAIN RUNS:")

print(f"\t{prefix}.coupPPE-hist.{nstring}")
print(f"\t{prefix}.offlPPE-ssp370.{nstring}")

print()

print(f"\trunscript.{prefix}.coupPPE-hist.{nstring}")
print(f"\trunscript.{prefix}.offlPPE-ssp370.{nstring}")

print("\nTEST RUNS:")

print(f"\t{prefix}.coupPPE-hist-nsteps2.000")
print(f"\t{prefix}.coupPPE-hist-ndays2.000")
print(f"\t{prefix}.coupPPE-hist-nmonths2.000")

print()

print(f"\trunscript.coupPPE-hist-2steps.000.{datetag}")
print(f"\trunscript.coupPPE-hist-2days.000.{datetag}")
print(f"\trunscript.coupPPE-hist-2months.000.{datetag}")

MAIN RUNS:
	f.e21.FHIST_BGC.f19_f19_mg17.coupPPE-hist.000
	f.e21.FHIST_BGC.f19_f19_mg17.offlPPE-ssp370.000

	runscript.f.e21.FHIST_BGC.f19_f19_mg17.coupPPE-hist.000
	runscript.f.e21.FHIST_BGC.f19_f19_mg17.offlPPE-ssp370.000

TEST RUNS:
	f.e21.FHIST_BGC.f19_f19_mg17.coupPPE-hist-nsteps2.000
	f.e21.FHIST_BGC.f19_f19_mg17.coupPPE-hist-ndays2.000
	f.e21.FHIST_BGC.f19_f19_mg17.coupPPE-hist-nmonths2.000

	runscript.coupPPE-hist-2steps.000.2025-07-03
	runscript.coupPPE-hist-2days.000.2025-07-03
	runscript.coupPPE-hist-2months.000.2025-07-03


# 5. Generate run scripts

In [2]:
# Claire's source code to check versions against
claire_srcdir = '/glade/u/home/czarakas/cesm_source/cesm_coupled_PPEn11'
print(f"Claire's CESM source code: {claire_srcdir}")
print('CESM tag:', end=' ')
!cd $claire_srcdir && git describe --tags
print('CLM  tag:', end=' ')
!cd $claire_srcdir/components/clm && git describe --tags
print('CAM  tag:', end=' ')
!cd $claire_srcdir/components/cam && git describe --tags

# Source code to use to run model
my_srcdir = '/glade/u/home/bbuchovecky/cesm_source/cesm_coupled_PPEn11'
print(f"\nmy CESM source code: {my_srcdir}")
print('CESM tag:', end=' ')
!cd $my_srcdir && git describe --tags
print('CLM  tag:', end=' ')
!cd $my_srcdir/components/clm && git describe --tags
print('CAM  tag:', end=' ')
!cd $my_srcdir/components/cam && git describe --tags

Claire's CESM source code: /glade/u/home/czarakas/cesm_source/cesm_coupled_PPEn11
CESM tag: cesm2.2.0
CLM  tag: branch_tags/PPE.n11_ctsm5.1.dev030
CAM  tag: cam_cesm2_2_rel_02

my CESM source code: /glade/u/home/bbuchovecky/cesm_source/cesm_coupled_PPEn11
CESM tag: cesm2.2.0
CLM  tag: branch_tags/PPE.n11_ctsm5.1.dev030
CAM  tag: cam_cesm2_2_rel_02


## 5.1. Default
!! I made a mistake with the naming of the default run and wrote `f19_f17_mg17` instead of `f19_f19_mg17`. This is fixed in the rest of the perturbed parameter runs.

In [ ]:
n = 0
my_casedescr = 'coupPPE-hist'

nstring = str(n).zfill(3)
cmps = "FHIST_BGC"
lcmps = cmps[0].lower()
vcesm = "e22"
# res = "f19_f19_mg17"
res = "f19_f17_mg17"
prefix = f"{lcmps}.{vcesm}.{cmps}.{res}"
my_casename = f'{prefix}.{my_casedescr}.{nstring}'

# Where you want to put the new folder for the case (where you change things like the 
# namelist, how long to run it, then do case.build, case.submit etc.)
# my_casedir = '/glade/u/home/bbuchovecky/cesm_runs/cases'
my_casedir = '/glade/u/home/bbuchovecky/cesm_runs/cases/coupPPE-hist/coupled_simulations'

# Where the output of the runs should be archived
# probably /glade/scratch/USERNAME/archive/
# my_archivedir = '/glade/derecho/scratch/bbuchovecky/archive'
my_archivedir = '/glade/derecho/scratch/bbuchovecky/archive/coupPPE-hist/coupled_simulations'

# Where to put the run folder (where log files will go, etc.)
# probably /glade/scratch/USERNAME
my_rundir = '/glade/derecho/scratch/bbuchovecky'

# Where you want to save the runscripts that this script will generate
# this can be anything, just make sure the folder exists!
runscript_path = '/glade/u/home/bbuchovecky/projects/cpl_ppe_co2/hist/code/03_run-scripts'

# Where to find ensemble member parameter files
paramfile_path = '/glade/u/home/bbuchovecky/projects/cpl_ppe_co2/hist/data/paramfiles'

# Where to find ensemble member namelist files
namelistmod_path = '/glade/u/home/bbuchovecky/projects/cpl_ppe_co2/hist/data/namelist_mods'

# The template to use to make the runscripts
template_file = './runscript.TEMPLATE'

print(f'\nCASE_DIR:    {my_casedir}')
print(f'ARCHIVE_DIR: {my_archivedir}')
print(f'RUN_DIR:     {my_rundir}')

this_paramfile = '/glade/u/home/bbuchovecky/projects/cpl_ppe_co2/hist/data/paramfiles/param.coupPPE-hist.000.nc'
this_namelist = '/glade/u/home/bbuchovecky/projects/cpl_ppe_co2/hist/data/namelist_mods/user_nl_clm_coupPPE-hist.000'

datetag = datetime.datetime.now().strftime('%y%m%d')
fout = f'{runscript_path}/runscript.{my_casedescr}.{nstring}.{datetag}'

notes = f'# {datetime.datetime.now().strftime("%y-%m-%dT%H:%M:%S")}\n# bbuchovecky'

f = open(template_file, 'r')
fdata = f.read()
f.close()

newdata =   fdata.replace('TEMPLATE_CASENAME',   my_casename)
newdata = newdata.replace('TEMPLATE_SRCDIR',     my_srcdir)
newdata = newdata.replace('TEMPLATE_CASEDIR',    my_casedir)
newdata = newdata.replace('TEMPLATE_ARCHDIR',    my_archivedir)
newdata = newdata.replace('TEMPLATE_RUNDIR',     my_rundir)
newdata = newdata.replace('TEMPLATE_PARAMFILE',  this_paramfile)
newdata = newdata.replace('TEMPLATE_NAMELIST',   this_namelist)
newdata = newdata.replace('TEMPLATE_FILENAME',   fout)
newdata = newdata.replace('# INSERT NOTES HERE', notes)

if f not in glob.glob(f'{runscript_path}/*'):
    f = open(fout, 'w')
    f.write(newdata)
    f.close()
else:
    print("file already exists")

# Make the script executable
!chmod +x $fout

for f in sorted(glob.glob(f'{runscript_path}/*')):
    print(f)


CASE_DIR:    /glade/u/home/bbuchovecky/cesm_runs/cases/cpl_ppe_co2/coupled_simulations
ARCHIVE_DIR: /glade/derecho/scratch/bbuchovecky/archive/cpl_ppe_co2/coupled_simulations
RUN_DIR:     /glade/derecho/scratch/bbuchovecky
/glade/u/home/bbuchovecky/projects/cpl_ppe_co2/hist/code/03_run-scripts/runscript.coupPPE-hist.000.250620
/glade/u/home/bbuchovecky/projects/cpl_ppe_co2/hist/code/03_run-scripts/runscript.coupPPE-hist.000.250622-continue
/glade/u/home/bbuchovecky/projects/cpl_ppe_co2/hist/code/03_run-scripts/runscript.coupPPE-hist.000.250623
/glade/u/home/bbuchovecky/projects/cpl_ppe_co2/hist/code/03_run-scripts/test_simulations


## 5.2. Perturbed

In [ ]:
crosswalk = pd.read_csv("coupPPE-hist_OAAT.csv", names=["casename", "param", "minmax"])

pdir = "/glade/u/home/bbuchovecky/projects/cpl_ppe_co2/hist/data/paramfiles"
ndir = "/glade/u/home/bbuchovecky/projects/cpl_ppe_co2/hist/data/namelist_mods"

cmps = "FHIST_BGC"
lcmps = cmps[0].lower()
vcesm = "e22"
res = "f19_f19_mg17"
prefix = f"{lcmps}.{vcesm}.{cmps}.{res}"

# Where you want to put the new folder for the case (where you change things like the 
# namelist, how long to run it, then do case.build, case.submit etc.)
# my_casedir = '/glade/u/home/bbuchovecky/cesm_runs/cases'
my_casedir = '/glade/u/home/bbuchovecky/cesm_runs/cases/coupPPE-hist/coupled_simulations'

# Where the output of the runs should be archived
# probably /glade/scratch/USERNAME/archive/
# my_archivedir = '/glade/derecho/scratch/bbuchovecky/archive'
my_archivedir = '/glade/derecho/scratch/bbuchovecky/archive/coupPPE-hist/coupled_simulations'

# Where to put the run folder (where log files will go, etc.)
# probably /glade/scratch/USERNAME
my_rundir = '/glade/derecho/scratch/bbuchovecky'

# Where you want to save the runscripts that this script will generate
# this can be anything, just make sure the folder exists!
runscript_path = '/glade/u/home/bbuchovecky/projects/cpl_ppe_co2/hist/code/03_run-scripts'

# Where to find ensemble member parameter files
paramfile_path = '/glade/u/home/bbuchovecky/projects/cpl_ppe_co2/hist/data/paramfiles'

# Where to find ensemble member namelist files
namelistmod_path = '/glade/u/home/bbuchovecky/projects/cpl_ppe_co2/hist/data/namelist_mods'

# The template to use to make the runscripts
template_file = './runscript.TEMPLATE'

print(f'\nCASE_DIR:    {my_casedir}')
print(f'ARCHIVE_DIR: {my_archivedir}')
print(f'RUN_DIR:     {my_rundir}')

for index, row in crosswalk.iterrows():
    my_casename = f'{prefix}.{row.casename}'

    this_paramfile = f"{pdir}/{row.casename}.nc"
    this_namelist = f"{ndir}/{row.casename}.txt"

    datetag = datetime.datetime.now().strftime('%y%m%d')
    fout = f'{runscript_path}/runscript.{row.casename}.{datetag}'

    notes = f'# {datetime.datetime.now().strftime("%y-%m-%dT%H:%M:%S")}\n# bbuchovecky'

    f = open(template_file, 'r')
    fdata = f.read()
    f.close()

    newdata =   fdata.replace('TEMPLATE_CASENAME',   my_casename)
    newdata = newdata.replace('TEMPLATE_SRCDIR',     my_srcdir)
    newdata = newdata.replace('TEMPLATE_CASEDIR',    my_casedir)
    newdata = newdata.replace('TEMPLATE_ARCHDIR',    my_archivedir)
    newdata = newdata.replace('TEMPLATE_RUNDIR',     my_rundir)
    newdata = newdata.replace('TEMPLATE_PARAMFILE',  this_paramfile)
    newdata = newdata.replace('TEMPLATE_NAMELIST',   this_namelist)
    newdata = newdata.replace('TEMPLATE_FILENAME',   fout)
    newdata = newdata.replace('# INSERT NOTES HERE', notes)

    # if f not in glob.glob(f'{runscript_path}/*'):
    #     f = open(fout, 'w')
    #     f.write(newdata)
    #     f.close()
    # else:
    #     print("file already exists")
    
    # # Make the script executable
    # !chmod +x $fout

    

!ls -lstr $runscript_path

for f in sorted(glob.glob(f'{runscript_path}/*')):
    print(f)


CASE_DIR:    /glade/u/home/bbuchovecky/cesm_runs/cases/coupPPE-hist/coupled_simulations
ARCHIVE_DIR: /glade/derecho/scratch/bbuchovecky/archive/coupPPE-hist/coupled_simulations
RUN_DIR:     /glade/derecho/scratch/bbuchovecky
total 34
32 -rwxr-xr-x 1 bbuchovecky uwas0155 11212 Jun 23 09:41 runscript.coupPPE-hist.000.250623
 1 -rwxr-xr-x 1 bbuchovecky uwas0155  3261 Jun 23 23:32 runscript.coupPPE-hist.000.250623-continue
 1 drwxr-xr-x 2 bbuchovecky uwas0155  4096 Jun 28 14:49 test_simulations
/glade/u/home/bbuchovecky/projects/cpl_ppe_co2/hist/code/03_run-scripts/runscript.coupPPE-hist.000.250623
/glade/u/home/bbuchovecky/projects/cpl_ppe_co2/hist/code/03_run-scripts/runscript.coupPPE-hist.000.250623-continue
/glade/u/home/bbuchovecky/projects/cpl_ppe_co2/hist/code/03_run-scripts/test_simulations
